In [1]:
import pandas as pd
import numpy as np
import napari
import pytest

from pathlib import Path
from aicsimageio import AICSImage, readers
from blimp.preprocessing.illumination_correction import IlluminationCorrection
import blimp.processing.quantify

03-Nov-24 22:40:46 - bfio.backends - WARNING  - Java backend is not available. This could be due to a missing dependency (jpype).


In [2]:
viewer = napari.Viewer()

In [3]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [4]:

_data_dir = Path("/Users/z3532965/src/blimp/tests/_data/")
intensity_image_2D = AICSImage(_data_dir / "datasets" / "synthetic_images" / "synthetic_intensity_image_TYX.tiff")
label_image_2D = AICSImage(_data_dir / "datasets" / "synthetic_images" / "synthetic_label_image_TYX.tiff")
intensity_image_3D = AICSImage(_data_dir / "datasets" / "synthetic_images" / "synthetic_intensity_image_TZYX.tiff")
label_image_3D = AICSImage(_data_dir / "datasets" / "synthetic_images" / "synthetic_label_image_TZYX.tiff")


``_measure_parent_object_label``

In [5]:
with pytest.raises(ValueError):
    # check for ValueError when assignment of measure objects to parent is ambiguous
    blimp.processing.quantify._measure_parent_object_label(
        label_image = label_image_2D,
        measure_object_index = 2,
        parent_object_index = 0,
        timepoint = 0)

res = blimp.processing.quantify._measure_parent_object_label(
    label_image = label_image_2D,
    measure_object_index = 1,
    parent_object_index = 0,
    timepoint = 0)

res.to_csv(_data_dir / "resources" / "measure_parent_label_2D_results_t_0.csv")

[viewer.add_labels(label_image_2D.get_image_data("YX",C=i),name=label_image_2D.channel_names[i]) for i in range(label_image_2D.dims.C)]


03-Nov-24 22:40:50 - blimp.processing.quantify - INFO     - ``label_image`` is 2D. Quantifying 2D features only.
03-Nov-24 22:40:50 - blimp.processing.quantify - INFO     - ``label_image`` is 2D. Quantifying 2D features only.


[<Labels layer 'Object1' at 0x3306599f0>,
 <Labels layer 'Object2' at 0x103942410>,
 <Labels layer 'Object3' at 0x33e2124a0>]

In [6]:
# check visually that this looks ok

# Convert DataFrame to dictionary for mapping
parent_labels_dict = dict(zip(res['label'], res['parent_label']))

# Replace values in the 2D array using np.vectorize with the dictionary
original_labels_array = label_image_2D.get_image_data("YX",C=1)
parent_labels_array = np.vectorize(parent_labels_dict.get)(original_labels_array, original_labels_array)
viewer.add_labels(parent_labels_array)


<Labels layer 'parent_labels_array' at 0x103943160>

3D

In [7]:
with pytest.raises(ValueError):
    # check for ValueError when assignment of measure objects to parent is ambiguous
    blimp.processing.quantify._measure_parent_object_label(
        label_image = label_image_3D,
        measure_object_index = 2,
        parent_object_index = 0,
        timepoint = 0)

res = blimp.processing.quantify._measure_parent_object_label(
    label_image = label_image_3D,
    measure_object_index = 1,
    parent_object_index = 0,
    timepoint = 0)

res.to_csv(_data_dir / "datasets" / "synthetic_images" / "measure_parent_label_3D_results_t_0.csv")

[viewer.add_labels(label_image_3D.get_image_data("ZYX",C=i),name=label_image_2D.channel_names[i]) for i in range(label_image_2D.dims.C)]


03-Nov-24 22:40:51 - blimp.processing.quantify - INFO     - ``label_image`` is 3D (200 Z-planes). Measuring parent in 3D.
03-Nov-24 22:40:51 - blimp.processing.quantify - INFO     - ``label_image`` is 3D (200 Z-planes). Measuring parent in 3D.


[<Labels layer 'Object1 [1]' at 0x3423e80d0>,
 <Labels layer 'Object2 [1]' at 0x35d371fc0>,
 <Labels layer 'Object3 [1]' at 0x35d76bee0>]

In [8]:
# check visually that this looks ok

# Convert DataFrame to dictionary for mapping
parent_labels_dict = dict(zip(res['label'], res['parent_label']))

# Replace values in the 2D array using np.vectorize with the dictionary
original_labels_array = label_image_3D.get_image_data("ZYX",C=1)
parent_labels_array = np.vectorize(parent_labels_dict.get)(original_labels_array, original_labels_array)
viewer.add_labels(parent_labels_array)

<Labels layer 'parent_labels_array [1]' at 0x33dd3e320>

``_quantify_single_timepoint_2D``

In [9]:

# no parent object
res_obj1 = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0)

res_obj1.Object1_intensity_max_Channel1.to_list() == [1000.,2000.,3000.,4000.]
res_obj1.Object1_area.to_list() == [100.0,81.0,900.0,961.0]

res_obj2 = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0)

res_obj2.count().label==100
list(np.unique(res_obj2.Object2_area)) == [4.0]

with pytest.raises(AttributeError):
    # check no parent label when parent_label is none
    parents = res_obj2.parent_label

# parent object
res_obj1_parent1 = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    parent_object = 0,
    timepoint = 0)

# check the parent object labels are correct when parent = measure
res_obj1_parent1.parent_label.to_list() == [1,2,3,4]

with pytest.raises(ValueError):
    # check for ValueError when assignment of measure objects to parent is ambiguous
    res_obj3_parent1 = blimp.processing.quantify._quantify_single_timepoint_2D(
        intensity_image = intensity_image_2D,
        label_image = label_image_2D,
        measure_object = 2,
        parent_object = 0,
        timepoint = 0)

# quantify object channel 1 relative to parent object channel 0
res_obj2_parent1 = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    parent_object = 0,
    timepoint = 0)

list(np.unique(res_obj2_parent1.parent_label_name)) == ["Object1"]

# check the number of objects in each parent object is correct
res_obj2_parent1.query("parent_label != 0").groupby('parent_label').size().to_list() == [5,4,21,20]


03-Nov-24 22:40:52 - blimp.processing.quantify - INFO     - ``label_image`` is 2D. Quantifying 2D features only.
03-Nov-24 22:40:52 - blimp.processing.quantify - INFO     - ``label_image`` is 2D. Quantifying 2D features only.
03-Nov-24 22:40:52 - blimp.processing.quantify - INFO     - ``label_image`` is 2D. Quantifying 2D features only.


True

In [10]:
# test intensity_channels
res_obj2_intensity1_str_input = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels="Channel1")

res_obj2_intensity1_list1_str_input = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=["Channel1"])

res_obj2_intensity1_list2_str_input = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=["Channel1","Channel2"])

res_obj2_intensity1_list_int_input = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=[0,1])

with pytest.raises(AttributeError):
    res_obj2_intensity1_str_input.Object2_intensity_min_Channel2
with pytest.raises(AttributeError):
    res_obj2_intensity1_list1_str_input.Object2_intensity_min_Channel2
with pytest.raises(AttributeError):
    res_obj2_intensity1_list2_str_input.Object2_intensity_min_Channel3

list(np.unique(res_obj2_intensity1_str_input.Object2_intensity_min_Channel1)) == [0,1000.,2000.,3000.,4000.]
list(np.unique(res_obj2_intensity1_list1_str_input.Object2_intensity_min_Channel1)) == [0,1000.,2000.,3000.,4000.]
list(np.unique(res_obj2_intensity1_list2_str_input.Object2_intensity_min_Channel2)) == [0,5000.]


True

In [11]:
# test texture_channels

# calculate_textures = False overrides texture channels provided
res_obj1_texture1_str_input_texture_false = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=False,
    texture_channels="Channel1")

with pytest.raises(KeyError):
    res_obj1_texture1_str_input_texture_false['Object1_Channel1_Haralick-angular-second-moment-1']

# calculate_textures = True with texture channels not specified calculates all
res_obj1_texture_none_texture_true = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=True)

len(res_obj1_texture_none_texture_true['Object1_Channel3_Haralick-diff-var-3'].to_list())==4

# calculate_textures = True with texture channels specified calculates only those specified
res_obj1_texture1_str_input_texture_true = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=True,
    texture_channels="Channel1")

res_obj1_texture1_str_input_texture_true['Object1_Channel1_Haralick-angular-second-moment-1'].to_list() == [1.,1.,1.,1.]
with pytest.raises(KeyError):
    res_obj1_texture1_str_input_texture_false['Object1_Channel3_Haralick-angular-second-moment-1']


In [12]:
len(res_obj1_texture_none_texture_true['Object1_Channel3_Haralick-diff-var-3'].to_list())==4

True

In [13]:
# test no border objects
res_no_border = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    intensity_channels="Channel1")

not any(res_no_border.Object1_is_border)

# test one border object (crop image to generate a border object)
res_border = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = AICSImage(
        intensity_image_2D.data[:,:,:,52:,52:],
        channel_names=intensity_image_2D.channel_names),
    label_image = AICSImage(
        label_image_2D.data[:,:,:,52:,52:],
        channel_names=label_image_2D.channel_names),
    measure_object = 0,
    timepoint = 0,
    intensity_channels="Channel1")

res_border.query("label==1").Object1_is_border[0]==True


True

``_quantify_single_timepoint_3D``

In [14]:
label_image_3D.channel_names
intensity_image_3D.channel_names

['Channel1', 'Channel2', 'Channel3']

In [15]:
# no parent object

res_obj1 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 0,
    timepoint = 0)

res_obj1['Object1-3D-MIP_area'].to_list()==[10.**2, 9.**2, 30.**2, 46.**2]
res_obj1['Object1_3D_area'].to_list()==[10.**3, 9.**3, 30.**3, 46.**3]
res_obj1['Object1-3D-Middle_perimeter'].to_list()==[10.*4-4, 9.*4-4, 30.*4-4, 46.*4-4]

res_obj2 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 1,
    timepoint = 0)

res_obj2.count().label==100
list(np.unique(res_obj2.Object2_3D_area)) == [2.**3]

with pytest.raises(AttributeError):
    # check no parent label when parent_label is none
    parents = res_obj2.parent_label

# parent object
res_obj1_parent1 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 0,
    parent_object = 0,
    timepoint = 0)

# check the parent object labels are correct when parent = measure
res_obj1_parent1.parent_label.to_list() == [1,2,3,4]

res_obj2_parent1 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image=intensity_image_3D,
    label_image=label_image_3D,
    measure_object=1,
    parent_object=0,
    timepoint=0
)

res_obj2_parent1.query("parent_label != 0").groupby('parent_label').size().to_list()

03-Nov-24 22:40:55 - blimp.processing.quantify - WARNING  - Detecting parent objects in 3D leads to ambiguity for object relationships in derived 2D features.
03-Nov-24 22:40:55 - blimp.processing.quantify - WARNING  - Derived 2D features (3D-MIP and 3D-Middle) are omitted for clarity.
03-Nov-24 22:40:55 - blimp.processing.quantify - INFO     - ``label_image`` is 3D (200 Z-planes). Measuring parent in 3D.
03-Nov-24 22:40:55 - blimp.processing.quantify - WARNING  - Detecting parent objects in 3D leads to ambiguity for object relationships in derived 2D features.
03-Nov-24 22:40:55 - blimp.processing.quantify - WARNING  - Derived 2D features (3D-MIP and 3D-Middle) are omitted for clarity.
03-Nov-24 22:40:56 - blimp.processing.quantify - INFO     - ``label_image`` is 3D (200 Z-planes). Measuring parent in 3D.


[7, 5, 15, 13]

In [16]:
res_obj2.columns

Index(['label', 'Object2_3D_centroid_0', 'Object2_3D_centroid_1',
       'Object2_3D_centroid_2', 'Object2_3D_area', 'Object2_3D_area_convex',
       'Object2_3D_axis_major_length', 'Object2_3D_axis_minor_length',
       'Object2_3D_extent', 'Object2_3D_feret_diameter_max',
       'Object2_3D_solidity', 'Object2_3D_intensity_mean_Channel1',
       'Object2_3D_intensity_max_Channel1',
       'Object2_3D_intensity_min_Channel1', 'Object2_3D_intensity_sd_Channel1',
       'Object2_3D_intensity_median_Channel1',
       'Object2_3D_intensity_mean_Channel2',
       'Object2_3D_intensity_max_Channel2',
       'Object2_3D_intensity_min_Channel2', 'Object2_3D_intensity_sd_Channel2',
       'Object2_3D_intensity_median_Channel2',
       'Object2_3D_intensity_mean_Channel3',
       'Object2_3D_intensity_max_Channel3',
       'Object2_3D_intensity_min_Channel3', 'Object2_3D_intensity_sd_Channel3',
       'Object2_3D_intensity_median_Channel3', 'Object2-3D-MIP_area',
       'Object2-3D-MIP_area_con

In [17]:
[viewer.add_labels(label_image_3D.get_image_data("ZYX",C=i),name=label_image_2D.channel_names[i]) for i in range(label_image_2D.dims.C)]


[<Labels layer 'Object1 [2]' at 0x35b2f2110>,
 <Labels layer 'Object2 [2]' at 0x35d0f8fa0>,
 <Labels layer 'Object3 [2]' at 0x35b2f0550>]

In [22]:
# parent object
res_obj1_parent1 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 0,
    parent_object = 0,
    timepoint = 0)

# check the parent object labels are correct when parent = measure
res_obj1_parent1.parent_label.to_list() == [1,2,3,4]

with pytest.raises(ValueError):
    # check for ValueError when assignment of measure objects to parent is ambiguous
    res_obj3_parent1 = blimp.processing.quantify._quantify_single_timepoint_3D(
        intensity_image = intensity_image_3D,
        label_image = label_image_3D,
        measure_object = 2,
        parent_object = 0,
        timepoint = 0)

# quantify object channel 1 relative to parent object channel 0
res_obj2_parent1 = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 1,
    parent_object = 0,
    timepoint = 0)

list(np.unique(res_obj2_parent1.parent_label_name)) == ["Object1"]

# check the number of objects in each parent object is correct
res_obj2_parent1.query("parent_label != 0").groupby('parent_label').size().to_list() == [7, 5, 15, 13]

03-Nov-24 22:42:55 - blimp.processing.quantify - WARNING  - Detecting parent objects in 3D leads to ambiguity for object relationships in derived 2D features.
03-Nov-24 22:42:55 - blimp.processing.quantify - WARNING  - Derived 2D features (3D-MIP and 3D-Middle) are omitted for clarity.
03-Nov-24 22:42:55 - blimp.processing.quantify - INFO     - ``label_image`` is 3D (200 Z-planes). Measuring parent in 3D.
03-Nov-24 22:42:56 - blimp.processing.quantify - WARNING  - Detecting parent objects in 3D leads to ambiguity for object relationships in derived 2D features.
03-Nov-24 22:42:56 - blimp.processing.quantify - WARNING  - Derived 2D features (3D-MIP and 3D-Middle) are omitted for clarity.
03-Nov-24 22:42:56 - blimp.processing.quantify - INFO     - ``label_image`` is 3D (200 Z-planes). Measuring parent in 3D.
03-Nov-24 22:42:56 - blimp.processing.quantify - WARNING  - Detecting parent objects in 3D leads to ambiguity for object relationships in derived 2D features.
03-Nov-24 22:42:56 - bl

True

In [ ]:
# test intensity_channels
res_obj2_intensity1_str_input = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels="Channel1")

res_obj2_intensity1_list1_str_input = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=["Channel1"])

res_obj2_intensity1_list2_str_input = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=["Channel1","Channel2"])

res_obj2_intensity1_list_int_input = blimp.processing.quantify._quantify_single_timepoint_3D(
    intensity_image = intensity_image_3D,
    label_image = label_image_3D,
    measure_object = 1,
    timepoint = 0,
    intensity_channels=[0,1])

with pytest.raises(AttributeError):
    res_obj2_intensity1_str_input.Object2_intensity_min_Channel2
with pytest.raises(AttributeError):
    res_obj2_intensity1_list1_str_input.Object2_intensity_min_Channel2
with pytest.raises(AttributeError):
    res_obj2_intensity1_list2_str_input.Object2_intensity_min_Channel3

list(np.unique(res_obj2_intensity1_str_input.Object2_3D_intensity_min_Channel1)) == [0,1000.,2000.,3000.,4000.]
list(np.unique(res_obj2_intensity1_list1_str_input.Object2_3D_intensity_min_Channel1)) == [0,1000.,2000.,3000.,4000.]
list(np.unique(res_obj2_intensity1_list2_str_input.Object2_3D_intensity_min_Channel2)) == [5000.,6000.]


In [25]:
res_obj2_intensity1_list_int_input.columns

Index(['label', 'Object2_3D_centroid_0', 'Object2_3D_centroid_1',
       'Object2_3D_centroid_2', 'Object2_3D_area', 'Object2_3D_area_convex',
       'Object2_3D_axis_major_length', 'Object2_3D_axis_minor_length',
       'Object2_3D_extent', 'Object2_3D_feret_diameter_max',
       'Object2_3D_solidity', 'Object2_3D_intensity_mean_Channel2',
       'Object2_3D_intensity_max_Channel2',
       'Object2_3D_intensity_min_Channel2', 'Object2_3D_intensity_sd_Channel2',
       'Object2_3D_intensity_median_Channel2',
       'Object2_3D_intensity_mean_Channel1',
       'Object2_3D_intensity_max_Channel1',
       'Object2_3D_intensity_min_Channel1', 'Object2_3D_intensity_sd_Channel1',
       'Object2_3D_intensity_median_Channel1', 'Object2-3D-MIP_area',
       'Object2-3D-MIP_area_convex', 'Object2-3D-MIP_axis_major_length',
       'Object2-3D-MIP_axis_minor_length', 'Object2-3D-MIP_eccentricity',
       'Object2-3D-MIP_extent', 'Object2-3D-MIP_feret_diameter_max',
       'Object2-3D-MIP_solidity

In [ ]:
list(np.unique(res_obj2_intensity1_str_input.Object2_3D_intensity_min_Channel1)) == [0,1000.,2000.,3000.,4000.]
list(np.unique(res_obj2_intensity1_list1_str_input.Object2_3D_intensity_min_Channel1)) == [0,1000.,2000.,3000.,4000.]
list(np.unique(res_obj2_intensity1_list2_str_input.Object2_3D_intensity_min_Channel2)) == [5000.,6000.]

True

In [ ]:
res_obj2_intensity1_str_input.columns

In [ ]:
# test texture_channels

# calculate_textures = False overrides texture channels provided
res_obj1_texture1_str_input_texture_false = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=False,
    texture_channels="Channel1")

with pytest.raises(KeyError):
    res_obj1_texture1_str_input_texture_false['Object1_Channel1_Haralick-angular-second-moment-1']

# calculate_textures = True with texture channels not specified calculates all
res_obj1_texture_none_texture_true = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=True)

len(res_obj1_texture_none_texture_true['Object1_Channel3_Haralick-diff-var-3'].to_list())==4

# calculate_textures = True with texture channels specified calculates only those specified
res_obj1_texture1_str_input_texture_true = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    calculate_textures=True,
    texture_channels="Channel1")

res_obj1_texture1_str_input_texture_true['Object1_Channel1_Haralick-angular-second-moment-1'].to_list() == [1.,1.,1.,1.]
with pytest.raises(KeyError):
    res_obj1_texture1_str_input_texture_false['Object1_Channel3_Haralick-angular-second-moment-1']


In [ ]:
len(res_obj1_texture_none_texture_true['Object1_Channel3_Haralick-diff-var-3'].to_list())==4

In [ ]:
# test no border objects
res_no_border = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = intensity_image_2D,
    label_image = label_image_2D,
    measure_object = 0,
    timepoint = 0,
    intensity_channels="Channel1")

not any(res_no_border.Object1_is_border)

# test one border object (crop image to generate a border object)
res_border = blimp.processing.quantify._quantify_single_timepoint_2D(
    intensity_image = AICSImage(
        intensity_image_2D.data[:,:,:,52:,52:],
        channel_names=intensity_image_2D.channel_names),
    label_image = AICSImage(
        label_image_2D.data[:,:,:,52:,52:],
        channel_names=label_image_2D.channel_names),
    measure_object = 0,
    timepoint = 0,
    intensity_channels="Channel1")

res_border.query("label==1").Object1_is_border


In [ ]:
np.max(label_image_2D.get_image_data('YX',C=2))

In [ ]:
features_non_aggregated = quantify(
    intensity_image=intensity_image,
    label_image=all_channels,
    parent_object="Nuclei",
    aggregate=False)

In [ ]:
len(features_non_aggregated)

In [ ]:
features_aggregated = quantify(
    intensity_image=intensity_image,
    label_image=all_channels,
    parent_object="Nuclei",
    aggregate=True)

In [ ]:
features_aggregated.columns